In [2]:
# Install required packages
!pip install -q flask flask-ngrok pyngrok pillow opencv-python-headless

# Configure ngrok
import getpass
from pyngrok import ngrok
import os
import numpy as np
import cv2
import tensorflow as tf
import uuid
from flask import Flask, request, jsonify, render_template_string, url_for, redirect
from werkzeug.utils import secure_filename
from flask_ngrok import run_with_ngrok
from google.colab import files

# Step 1: Configure ngrok (you need to sign up for an account and get a token)
print("Enter your ngrok auth token (sign up at https://ngrok.com if you don't have one):")
auth_token = getpass.getpass('Enter your ngrok authtoken: ')
ngrok.set_auth_token(auth_token)

print("\n----- Model Setup -----")
print("Please upload your model file (best_cnn_model.h5):")

if not os.path.exists('/content/best_cnn_model.h5'):
    print("Please upload your best_cnn_model.h5 file:")
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f'Uploaded {filename}')
else:
    print("Model file already exists")

# Step 3: Create directories
UPLOAD_FOLDER = 'static/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
print(f"Created upload directory at {UPLOAD_FOLDER}")

# Step 4: Define image processing and detection functions
print("\n----- Loading Model -----")
model = tf.keras.models.load_model('best_cnn_model.h5')
print("Model loaded successfully!")

# Define the class names
CATEGORIES = {
0: "ball",
1: "circle cage",
2: "cube",
3: "cylinder",
4: "human body",
5: "metal bucket",
6: "plane",
7: "rov",
8: "square cage",
9: "tyre"
}

# Image size that the model expects
IMAGE_SIZE = (64, 64)  # Update this based on your model's input size

def preprocess_image(image_path):
    """Preprocess the image for model prediction"""
    # Read image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Store original dimensions for scaling back the bounding boxes
    orig_h, orig_w = img.shape[:2]

    # Resize to expected input size
    img_resized = cv2.resize(img, IMAGE_SIZE)

    # Normalize pixel values
    img_normalized = img_resized.astype(np.float32) / 255.0

    # Add batch dimension
    img_batch = np.expand_dims(img_normalized, axis=0)

    return img_batch, img, orig_h, orig_w

def detect_objects(image_path, upload_folder):
    """Detect objects in the image using the loaded model"""
    # Preprocess image
    img_batch, original_img, orig_h, orig_w = preprocess_image(image_path)

    # Make prediction
    predictions = model.predict(img_batch)

    # Extract bounding box and class predictions
    # The exact structure depends on your model's output
    if isinstance(predictions, list):
        # If model returns multiple outputs
        bbox_pred, class_pred = predictions
        bbox = bbox_pred[0]  # First item in batch
        class_probs = class_pred[0]  # First item in batch
    else:
        # If model returns a single output, adjust accordingly
        bbox = predictions[0, :4]
        class_probs = predictions[0, 4:]

    # Get predicted class
    class_id = np.argmax(class_probs)
    class_name = CATEGORIES.get(class_id, f"Unknown ({class_id})")
    confidence = float(class_probs[class_id])

    # Scale bounding box back to original image dimensions
    x, y, w, h = bbox
    x = x * orig_w
    y = y * orig_h
    w = w * orig_w
    h = h * orig_h

    # Draw bounding box on the image
    output_img = original_img.copy()
    cv2.rectangle(output_img,
                  (int(x), int(y)),
                  (int(x + w), int(y + h)),
                  (0, 255, 0), 2)

    # Add label
    label = f"{class_name}: {confidence:.2f}"
    cv2.putText(output_img, label,
                (int(x), int(y - 10)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the annotated image
    output_filename = f"result_{uuid.uuid4().hex}.jpg"
    output_path = os.path.join(upload_folder, output_filename)
    cv2.imwrite(output_path, cv2.cvtColor(output_img, cv2.COLOR_RGB2BGR))

    result = {
        "class_name": class_name,
        "confidence": confidence,
        "bbox": [float(x), float(y), float(w), float(h)],
        "result_image": output_filename
    }

    return result

# Step 5: Create Flask Application
app = Flask(__name__, static_folder='static')
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max upload size
run_with_ngrok(app)  # Start ngrok when app is run

# HTML Template with CSS
main_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Underwater Object Detection</title>
    <style>
        :root {
            --primary: #0077b6;
            --secondary: #48cae4;
            --accent: #00b4d8;
            --background: #03045e;
            --text: #caf0f8;
            --glass: rgba(255, 255, 255, 0.1);
            --coral: #ff9a8b;
            --dark-blue: #023e8a;
        }

        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }

        body {
            background: linear-gradient(135deg, var(--background), var(--dark-blue));
            color: var(--text);
            line-height: 1.6;
            min-height: 100vh;
            padding: 20px;
        }

        .glass-panel {
            background: var(--glass);
            backdrop-filter: blur(10px);
            border-radius: 15px;
            border: 1px solid rgba(255, 255, 255, 0.2);
            box-shadow: 0 8px 32px 0 rgba(0, 0, 0, 0.3);
            padding: 20px;
            margin-bottom: 25px;
        }

        .container {
            max-width: 1000px;
            margin: 0 auto;
            padding: 20px;
        }

        header {
            text-align: center;
            margin-bottom: 40px;
        }

        .logo {
            display: flex;
            align-items: center;
            justify-content: center;
            margin-bottom: 15px;
        }

        .logo h1 {
            font-size: 2.5rem;
            margin-left: 15px;
            background: linear-gradient(to right, var(--secondary), var(--coral));
            -webkit-background-clip: text;
            background-clip: text;
            color: transparent;
        }

        .subtitle {
            font-size: 1.2rem;
            color: var(--secondary);
            opacity: 0.8;
            margin-bottom: 10px;
        }

        main {
            display: flex;
            flex-direction: column;
            gap: 30px;
        }

        section {
            margin-bottom: 30px;
            padding: 25px;
            border-radius: 15px;
            background: rgba(3, 4, 94, 0.5);
            box-shadow: 0 4px 15px rgba(0, 0, 0, 0.2);
        }

        section h2 {
            margin-bottom: 20px;
            font-size: 1.8rem;
            color: var(--secondary);
            display: flex;
            align-items: center;
            gap: 10px;
        }

        section p {
            margin-bottom: 20px;
            font-size: 1.1rem;
            color: var(--text);
            opacity: 0.9;
        }

        .upload-container {
            margin-top: 20px;
            display: flex;
            flex-direction: column;
        }

        .upload-area {
            border: 2px dashed var(--secondary);
            border-radius: 10px;
            padding: 40px 20px;
            text-align: center;
            background: rgba(0, 119, 182, 0.1);
            transition: all 0.3s ease;
            cursor: pointer;
        }

        .upload-area:hover {
            background: rgba(0, 119, 182, 0.2);
            border-color: var(--accent);
            transform: translateY(-5px);
        }

        .upload-area.dragging {
            background: rgba(0, 180, 216, 0.3);
            border-color: var(--accent);
        }

        .upload-area i {
            font-size: 3rem;
            color: var(--secondary);
            margin-bottom: 15px;
            display: block;
        }

        .upload-area p {
            font-size: 1.2rem;
            margin-bottom: 15px;
        }

        .browse-btn {
            background: linear-gradient(135deg, var(--accent), var(--primary));
            color: white;
            border: none;
            padding: 12px 25px;
            border-radius: 25px;
            font-size: 1rem;
            font-weight: bold;
            cursor: pointer;
            transition: all 0.3s ease;
            box-shadow: 0 4px 10px rgba(0, 0, 0, 0.2);
            display: inline-block;
        }

        .browse-btn:hover {
            transform: translateY(-2px);
            box-shadow: 0 6px 15px rgba(0, 0, 0, 0.3);
        }

        footer {
            text-align: center;
            margin-top: 50px;
            padding-top: 20px;
            border-top: 1px solid rgba(255, 255, 255, 0.1);
        }

        footer p {
            font-size: 0.9rem;
            opacity: 0.7;
        }

        /* Results Styling */
        .results-section {
            display: none;
        }

        .results-container {
            display: flex;
            flex-wrap: wrap;
            gap: 30px;
        }

        .result-image {
            flex: 1;
            min-width: 300px;
            border-radius: 10px;
            overflow: hidden;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.3);
        }

        .result-image img {
            width: 100%;
            height: auto;
            display: block;
        }

        .result-details {
            flex: 1;
            min-width: 300px;
            display: flex;
            flex-direction: column;
            gap: 20px;
        }

        .detail-card {
            padding: 15px;
            border-radius: 10px;
        }

        .detail-card h3 {
            font-size: 1.2rem;
            margin-bottom: 10px;
            color: var(--secondary);
            display: flex;
            align-items: center;
            gap: 8px;
        }

        .detail-card p {
            font-size: 1.3rem;
            color: white;
            font-weight: 500;
            margin: 0;
        }

        .confidence-meter {
            height: 25px;
            background: rgba(255, 255, 255, 0.1);
            border-radius: 15px;
            overflow: hidden;
            position: relative;
            margin-top: 10px;
        }

        .confidence-bar {
            height: 100%;
            background: linear-gradient(135deg, var(--accent), var(--primary));
            border-radius: 15px;
            width: 0;
            transition: width 1s ease-in-out;
        }

        .confidence-meter span {
            position: absolute;
            top: 50%;
            left: 50%;
            transform: translate(-50%, -50%);
            color: white;
            font-weight: bold;
            text-shadow: 0 0 3px rgba(0, 0, 0, 0.5);
        }

        .new-detection-btn {
            background: linear-gradient(135deg, var(--secondary), var(--accent));
            color: white;
            border: none;
            padding: 12px 25px;
            border-radius: 25px;
            font-size: 1rem;
            font-weight: bold;
            cursor: pointer;
            transition: all 0.3s ease;
            box-shadow: 0 4px 10px rgba(0, 0, 0, 0.2);
            display: inline-block;
            margin-top: 20px;
            align-self: flex-start;
            text-decoration: none;
        }

        .new-detection-btn:hover {
            transform: translateY(-2px);
            box-shadow: 0 6px 15px rgba(0, 0, 0, 0.3);
        }

        /* Responsive Design */
        @media (max-width: 768px) {
            .logo h1 {
                font-size: 2rem;
            }

            .results-container {
                flex-direction: column;
            }

            .result-image, .result-details {
                width: 100%;
            }

            .upload-area {
                padding: 30px 15px;
            }

            section {
                padding: 20px 15px;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <header>
            <div class="logo">
                <h1>Underwater Object Detection</h1>
            </div>
            <p class="subtitle">AI-powered detection for sonar imagery</p>
        </header>

        <main>
            <section class="upload-section glass-panel">
                <h2>Upload Sonar Image</h2>
                <p>Upload an underwater sonar image to detect objects with our deep learning model</p>

                <div class="upload-container">
                    <form action="/detect" method="post" enctype="multipart/form-data" id="upload-form">
                        <div class="upload-area" id="upload-area" onclick="document.getElementById('file-input').click()">
                            <img src="https://img.icons8.com/ios/100/caf0f8/upload.png" alt="Upload" width="60">
                            <p>Drag & drop an image or click to browse</p>
                            <input type="file" id="file-input" name="file" accept="image/*" style="display: none;" onchange="showFileName()">
                            <button type="button" class="browse-btn">Browse Files</button>
                            <p id="file-name-display" style="display: none;"></p>
                        </div>
                        <button type="submit" class="browse-btn" style="margin-top: 20px; display: none;" id="detect-btn">Detect Objects</button>
                    </form>
                </div>
            </section>

            {% if result %}
            <section class="results-section glass-panel" id="results-section" style="display: block;">
                <h2>Detection Results</h2>

                <div class="results-container">
                    <div class="result-image">
                        <img src="{{ url_for('static', filename='uploads/' + result.result_image) }}" alt="Detection Result">
                    </div>
                    <div class="result-details">
                        <div class="detail-card glass-panel">
                            <h3>Detected Object</h3>
                            <p>{{ result.class_name }}</p>
                        </div>
                        <div class="detail-card glass-panel">
                            <h3>Confidence Score</h3>
                            <div class="confidence-meter">
                                <div class="confidence-bar" style="width: {{ result.confidence * 100 }}%;"></div>
                                <span>{{ (result.confidence * 100)|round(1) }}%</span>
                            </div>
                        </div>
                        <div class="detail-card glass-panel">
                            <h3>Bounding Box</h3>
                            <p>x: {{ result.bbox[0]|round(2) }}, y: {{ result.bbox[1]|round(2) }}, w: {{ result.bbox[2]|round(2) }}, h: {{ result.bbox[3]|round(2) }}</p>
                        </div>
                        <a href="/" class="new-detection-btn">New Detection</a>
                    </div>
                </div>
            </section>
            {% endif %}
        </main>

        <footer>
            <p>Underwater Object Detection using Deep Learning &copy; 2023</p>
            <p>Powered by TensorFlow and Flask</p>
        </footer>
    </div>

    <script>
        function showFileName() {
            const fileInput = document.getElementById('file-input');
            const fileNameDisplay = document.getElementById('file-name-display');
            const detectBtn = document.getElementById('detect-btn');

            if (fileInput.files.length > 0) {
                fileNameDisplay.textContent = 'Selected file: ' + fileInput.files[0].name;
                fileNameDisplay.style.display = 'block';
                detectBtn.style.display = 'inline-block';
            } else {
                fileNameDisplay.style.display = 'none';
                detectBtn.style.display = 'none';
            }
        }

        // Drag and drop functionality
        const uploadArea = document.getElementById('upload-area');

        ['dragenter', 'dragover', 'dragleave', 'drop'].forEach(eventName => {
            uploadArea.addEventListener(eventName, preventDefaults, false);
        });

        function preventDefaults(e) {
            e.preventDefault();
            e.stopPropagation();
        }

        ['dragenter', 'dragover'].forEach(eventName => {
            uploadArea.addEventListener(eventName, highlight, false);
        });

        ['dragleave', 'drop'].forEach(eventName => {
            uploadArea.addEventListener(eventName, unhighlight, false);
        });

        function highlight() {
            uploadArea.classList.add('dragging');
        }

        function unhighlight() {
            uploadArea.classList.remove('dragging');
        }

        uploadArea.addEventListener('drop', handleDrop, false);

        function handleDrop(e) {
            const dt = e.dataTransfer;
            const files = dt.files;
            const fileInput = document.getElementById('file-input');

            fileInput.files = files;
            showFileName();
        }
    </script>
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(main_template)

@app.route('/detect', methods=['POST'])
def detect():
    if 'file' not in request.files:
        return redirect(url_for('index'))

    file = request.files['file']

    if file.filename == '':
        return redirect(url_for('index'))

    if file:
        # Save the uploaded file
        filename = secure_filename(f"upload_{uuid.uuid4().hex}{os.path.splitext(file.filename)[1]}")
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        try:
            # Process the image and get results
            result = detect_objects(filepath, app.config['UPLOAD_FOLDER'])

            # Render the template with the results
            return render_template_string(main_template, result=result)
        except Exception as e:
            print(f"Error: {str(e)}")
            return redirect(url_for('index'))

# Step 6: Run the App (this is the last line to execute in Colab)
print("\n----- Starting Web Application -----")
print("Click the ngrok link below to access your web application")
app.run()

Enter your ngrok auth token (sign up at https://ngrok.com if you don't have one):
Enter your ngrok authtoken: ··········

----- Model Setup -----
Please upload your model file (best_cnn_model.h5):
Model file already exists
Created upload directory at static/uploads

----- Loading Model -----


Model loaded successfully!

----- Starting Web Application -----
Click the ngrok link below to access your web application
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            